# Set up

In [ ]:
!pip install pycaret

In [ ]:
import numpy as np; np.random.seed(42)
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import pycaret
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!mkdir 'Data'
!unzip 'drive/MyDrive/Kaggle/electric-power-consumption.zip' -d 'Data'

Mounted at /content/drive
Archive:  drive/MyDrive/Kaggle/electric-power-consumption.zip
  inflating: Data/PowerCons_TEST.csv  
  inflating: Data/PowerCons_TRAIN.csv  
  inflating: Data/README.md          
  inflating: Data/forecast_submission_example.csv  


# Prepare data and predict

In [ ]:
train = pd.read_csv('/content/Data/PowerCons_TRAIN.csv')
test = pd.read_csv('/content/Data/PowerCons_TEST.csv')

ตอนนี้พยายามแปลงข้อมูล train set ให้เหมือน test set 

In [ ]:
half_train=train.iloc[:,1:73]
another_half_train=train.iloc[:,73:len(train.columns)-2]
half_train['kW-12:00-23:50']=another_half_train.sum(axis=1)
half_train['Class']=train['Class'].astype(int)
half_train=half_train.rename(columns={'kW-12:00-23:50':'label'})
#half_train['Class']=half_train['Class'].map({1:'Hot',2:'Cold'})
half_train.head()

,00:00,00:10,00:20,00:30,00:40,00:50,01:00,01:10,01:20,01:30,01:40,01:50,02:00,02:10,02:20,02:30,02:40,02:50,03:00,03:10,03:20,03:30,03:40,03:50,04:00,04:10,04:20,04:30,04:40,04:50,05:00,05:10,05:20,05:30,05:40,05:50,06:00,06:10,06:20,06:30,06:40,06:50,07:00,07:10,07:20,07:30,07:40,07:50,08:00,08:10,08:20,08:30,08:40,08:50,09:00,09:10,09:20,09:30,09:40,09:50,10:00,10:10,10:20,10:30,10:40,10:50,11:00,11:10,11:20,11:30,11:40,11:50,label,Class
0,0.44,0.40,0.36,0.35,0.35,0.38,0.36,0.35,0.35,0.38,0.38,0.38,0.42,0.45,0.43,0.37,0.66,1.13,1.35,1.09,0.75,0.41,0.37,0.33,0.33,0.40,0.40,0.39,0.40,0.43,0.40,0.35,0.31,0.33,0.33,0.33,0.30,0.30,0.33,0.36,1.10,1.83,1.93,2.04,2.04,1.93,1.93,1.53,1.36,1.19,2.25,3.31,3.08,2.85,2.04,1.24,1.26,1.29,1.31,1.34,1.79,2.25,1.73,1.21,1.12,0.97,0.82,0.47,0.11,0.16,0.22,0.62,41.30,1
1,0.14,0.19,0.19,0.16,0.16,0.18,0.17,0.16,0.17,0.17,0.16,0.18,0.18,0.14,0.13,0.16,0.19,0.14,0.15,0.18,0.18,0.15,0.16,0.19,0.17,0.16,0.16,0.17,0.16,0.18,0.17,0.14,0.15,0.17,0.18,0.10,0.37,0.64,0.89,1.14,1.19,1.24,0.68,0.12,0.16,0.17,0.16,0.18,0.16,0.14,0.15,0.16,0.16,0.13,0.16,0.14,0.12,0.14,0.17,0.18,0.16,0.16,0.16,0.16,0.17,0.22,0.25,0.25,0.27,0.29,0.17,0.13,27.79,1
2,0.34,0.34,0.34,0.26,0.18,0.83,1.48,1.36,0.95,0.55,0.42,0.29,0.26,0.24,0.24,0.25,0.23,0.25,0.26,0.25,0.25,0.26,0.28,0.30,0.33,0.31,0.30,0.27,0.25,0.22,0.20,0.22,0.24,0.22,0.19,0.22,0.24,0.24,0.25,1.79,3.33,2.94,2.54,2.54,1.58,1.58,1.46,1.34,1.61,1.79,1.96,1.97,1.63,1.29,1.31,1.34,1.32,1.30,1.28,1.26,1.24,1.21,0.92,0.64,0.45,0.26,0.23,0.24,0.26,0.25,0.25,0.29,51.04,1
3,1.70,1.50,1.30,3.40,5.50,4.15,2.80,2.67,2.53,2.49,2.45,2.41,1.64,0.88,0.53,0.19,0.17,0.16,0.16,0.22,0.29,0.19,0.14,0.15,0.21,0.26,0.26,0.22,0.21,0.24,0.26,0.29,0.22,0.17,0.13,0.19,0.24,0.16,0.13,0.17,0.21,0.26,0.30,0.99,1.67,1.52,1.36,1.38,1.38,1.39,1.41,1.38,1.35,1.34,1.33,1.33,1.31,1.30,0.89,0.49,0.31,0.14,0.20,0.23,0.22,0.23,0.28,0.31,0.30,0.28,0.34,0.41,67.77,1
4,1.78,1.59,1.39,1.45,1.45,1.41,1.40,1.45,1.46,1.40,0.85,0.30,0.34,0.38,0.33,0.33,0.39,0.35,0.31,0.31,0.35,0.38,0.38,0.33,0.33,0.38,0.39,0.35,0.33,0.36,0.38,0.37,0.34,0.34,0.37,0.37,0.35,0.35,0.37,0.38,0.36,0.35,0.36,0.36,0.36,0.35,0.37,0.37,0.35,0.34,0.36,0.38,0.37,0.39,0.41,1.08,1.75,1.68,1.66,1.65,1.50,1.34,1.55,1.75,1.68,1.60,1.48,1.41,1.38,1.33,1.31,1.34,31.31,1


สร้าง model สำหรับทำนาย class เนื่องจากข้อมูล test set ไม่มี class

In [ ]:
classification_model=pycaret.classification.setup(data=half_train, target='Class', train_size=0.8,
               numeric_features=list(half_train.drop(['label','Class'],axis=1).columns),
               ignore_features=['label'],
               normalize=True,
               silent=True)

best_class_model=pycaret.classification.compare_models(sort='Accuracy',turbo=False)

AttributeError: ignored

In [ ]:
test_nodate=test.drop('Day',axis=1)
test_class=pycaret.classification.predict_model(best_class_model, data=test_nodate)['Label']

AttributeError: ignored

In [ ]:
def rmse(y,predict):
  return np.sqrt((np.sum((y-predict)**2))/len(y))

In [ ]:
reg_model=pycaret.regression.setup(data=diff_train, target='label', train_size=0.8,
               numeric_features=list(diff_train.drop(['label'],axis=1).columns),
               categorical_features=['Class'],
               normalize=True,
               silent=True)

best_reg_model=pycaret.regression.compare_models(sort='RMSE',turbo=False)

In [ ]:
#hot_train=diff_train.loc[diff_train.Class=='Hot',:].drop('Class',axis=1)
#cold_train=diff_train.loc[diff_train.Class=='Cold',:].drop('Class',axis=1)
#print(hot_train.head())
#print(cold_train.head())

     t0    t1    t2    t3    t4    t5  ...   t66   t67   t68   t69   t70  label
0 -0.04 -0.04 -0.01  0.00  0.03 -0.02  ... -0.35 -0.36  0.05  0.06  0.40  41.30
1  0.05  0.00 -0.03  0.00  0.02 -0.01  ...  0.00  0.02  0.02 -0.12 -0.04  27.79
2  0.00  0.00 -0.08 -0.08  0.65  0.65  ...  0.01  0.02 -0.01  0.00  0.04  51.04
3 -0.20 -0.20  2.10  2.10 -1.35 -1.35  ...  0.03 -0.01 -0.02  0.06  0.07  67.77
4 -0.19 -0.20  0.06  0.00 -0.04 -0.01  ... -0.07 -0.03 -0.05 -0.02  0.03  31.31

[5 rows x 72 columns]
      t0    t1    t2    t3    t4    t5  ...   t66   t67   t68   t69   t70   label
30 -0.02  0.00  0.02 -0.02 -0.03 -0.02  ...  0.05  0.05 -0.03  0.00  0.03   97.17
31  0.00 -1.10  0.00 -0.01 -0.02 -0.03  ...  0.01  0.05  0.03 -0.01  0.00   75.32
32  0.19  0.20  0.37  0.09  0.09 -0.01  ...  0.00 -0.38  0.00  0.08 -0.51  178.64
33  0.04  0.05  0.05 -0.02 -0.05 -0.04  ... -0.03  0.02  0.07 -0.02 -0.03   84.48
34  0.06  0.03 -0.06  0.01  0.06  0.01  ... -0.29 -0.28 -0.11 -0.03  0.03  135.57

[5 r

In [ ]:
#hot_reg_model=pycaret.regression.setup(data=hot_train, target='label', train_size=0.8,
               #numeric_features=list(hot_train.drop(['label'],axis=1).columns),
               #normalize=True,
               #silent=True)

#best_hot_reg_model=pycaret.regression.compare_models(sort='RMSE',turbo=False)

#cold_reg_model=pycaret.regression.setup(data=cold_train, target='label', train_size=0.8,
               #numeric_features=list(cold_train.drop(['label'],axis=1).columns),
               #normalize=True,
               #silent=True)

#best_cold_reg_model=pycaret.regression.compare_models(sort='RMSE',turbo=False)

In [ ]:
diff_test=pd.DataFrame()

first=list(test_nodate.columns)[:-1]
second=list(test_nodate.columns)[1:]
for t1,t2 in zip(first,second):
  diff_test=pd.concat([diff_test,test_nodate[t2]-test_nodate[t1]],axis=1)
diff_test.columns=['t'+str(i) for i in range(len(test_nodate.columns)-1)]
diff_test=pd.concat([diff_test,test_class],axis=1)
diff_test=diff_test.rename(columns={'Label':'Class'})
diff_test

,t0,t1,t2,t3,t4,t5,t6,t7,t8,t9,t10,t11,t12,t13,t14,t15,t16,t17,t18,t19,t20,t21,t22,t23,t24,t25,t26,t27,t28,t29,t30,t31,t32,t33,t34,t35,t36,t37,t38,t39,t40,t41,t42,t43,t44,t45,t46,t47,t48,t49,t50,t51,t52,t53,t54,t55,t56,t57,t58,t59,t60,t61,t62,t63,t64,t65,t66,t67,t68,t69,t70,Class
0,-0.03,-0.01,-0.01,0.00,0.01,0.01,0.49,0.48,0.06,0.06,-0.57,-0.56,0.00,0.06,-0.01,-0.10,0.09,0.00,-0.01,-0.01,-0.01,-0.02,0.01,0.01,-0.06,0.08,0.07,-0.07,-0.08,0.03,0.03,-0.02,-0.02,0.01,0.01,0.03,0.02,-0.03,-0.04,0.78,0.78,-0.33,0.27,0.27,0.38,0.37,0.00,-1.23,0.00,-0.14,-0.03,-0.04,0.02,-0.03,-0.03,-0.01,0.00,0.04,0.03,-0.01,-0.01,-0.50,-0.50,0.00,-0.05,0.00,0.00,0.11,0.00,0.74,0.74,Cold
1,0.00,0.00,0.00,0.04,0.00,-0.04,-0.03,0.02,0.01,0.02,-0.01,-0.01,0.00,0.03,0.00,-0.04,-0.05,0.00,0.01,0.02,0.03,-0.03,0.02,0.02,-0.05,-0.05,0.00,0.00,0.06,0.06,-0.07,-0.07,0.00,0.06,0.00,0.02,0.02,0.00,0.00,1.20,1.20,-0.53,-0.52,0.02,0.02,-0.42,-0.42,0.65,0.66,-0.25,-0.25,0.09,0.01,0.02,-0.28,-0.10,-0.09,0.09,-0.05,-0.06,0.00,0.01,0.00,0.00,-0.21,-0.36,-0.35,0.00,-0.06,0.00,0.58,Hot
2,-0.28,-0.28,-0.64,-0.65,-0.05,0.00,0.01,0.00,-0.03,-0.02,0.02,0.01,0.00,-0.01,0.00,0.03,0.00,-0.04,-0.04,0.01,0.01,0.02,0.02,0.00,0.10,0.00,-0.07,-0.08,0.00,0.00,0.04,0.04,0.00,-0.03,0.00,0.02,0.01,0.26,0.26,0.77,0.53,0.52,0.02,0.02,0.04,0.03,0.02,0.08,0.09,-0.62,-0.63,0.41,-0.19,-0.19,-0.22,-0.23,0.09,0.03,0.03,0.00,-0.07,0.00,-0.01,-0.02,-0.04,0.00,0.11,0.00,0.08,0.09,0.32,Cold
3,-0.43,-0.36,-0.37,-0.65,-0.11,-0.11,-0.16,-0.28,-0.28,-0.38,-0.19,-0.20,0.02,0.01,0.00,0.01,0.02,0.00,0.00,0.00,0.01,0.01,0.02,-0.04,-0.03,0.02,0.01,-0.04,-0.04,0.01,0.03,0.03,-0.01,-0.01,0.03,-0.01,-0.01,-0.01,-0.01,0.02,0.01,-0.03,-0.04,0.23,0.22,0.28,0.29,0.00,0.06,0.00,0.00,0.85,0.85,-1.30,-1.30,1.00,0.99,-0.91,-0.92,0.42,0.43,-0.02,-0.01,1.28,1.28,-0.83,-0.83,0.50,0.49,-0.06,-0.05,Hot
4,-0.11,-0.10,-0.05,0.01,0.04,0.03,-0.01,-0.01,-0.02,-0.01,-0.01,-0.01,-0.04,0.55,0.56,-0.06,-0.41,-0.41,-0.13,-0.07,0.01,0.03,-0.02,-0.02,0.02,0.01,-0.01,0.01,-0.02,0.00,0.03,0.00,-0.05,0.01,0.04,-0.01,-0.03,0.01,0.02,0.00,0.48,0.48,-0.37,-0.37,0.23,0.22,0.23,0.22,-0.11,-0.10,-0.24,-0.24,-0.25,-0.24,0.03,0.01,0.00,0.01,-0.03,0.00,0.03,0.00,-0.04,0.01,0.03,0.00,-0.02,0.00,0.01,0.00,0.01,Hot
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,0.00,-0.01,-0.03,-0.01,0.00,-0.03,-0.01,0.04,0.04,-0.02,0.01,0.01,-0.03,-0.04,0.02,0.02,0.03,-0.01,-0.01,-0.01,-0.02,0.00,0.00,0.01,0.03,0.03,-0.03,-0.03,-0.01,0.00,0.01,0.00,-0.03,-0.02,0.06,0.06,-0.06,-0.06,0.30,0.29,0.25,0.25,0.06,0.07,-0.59,-0.59,-0.01,0.00,0.04,0.03,-0.01,0.00,0.02,-0.05,-0.06,0.04,0.04,0.03,-0.01,-0.02,0.01,-0.01,-0.02,-0.02,-0.01,0.03,0.04,-0.02,-0.01,0.01,0.02,Hot
176,-0.09,-0.08,-0.26,-0.27,-0.26,-0.26,0.02,0.02,-0.02,-0.04,-0.02,0.03,0.00,-0.01,0.07,0.01,0.02,-0.05,-0.05,-0.01,0.03,0.01,0.01,-0.02,-0.04,0.03,0.04,-0.05,0.00,0.04,0.03,-0.02,-0.03,0.00,0.02,-0.01,-0.03,0.00,0.05,0.02,-0.01,-0.02,-0.02,0.03,-0.02,-0.02,0.00,0.00,0.01,0.00,-0.02,0.01,0.02,0.00,-0.02,0.00,0.01,-0.01,-0.02,0.01,0.01,0.01,0.01,-0.04,0.56,0.55,-0.03,-0.03,0.03,-0.24,-0.24,Hot
177,0.00,0.00,0.01,0.00,0.01,0.00,0.00,0.00,-0.02,-0.01,-0.02,-0.01,0.01,0.01,-0.02,-0.01,0.02,0.03,0.01,0.00,0.00,-0.01,-0.02,-0.01,0.02,0.03,-0.02,-0.02,-0.04,0.00,1.22,0.00,-0.10,-0.43,-0.43,-0.13,-0.12,0.04,0.03,0.07,0.34,0.34,-0.11,-0.11,0.40,0.40,0.21,0.21,-0.36,-0.35,0.01,0.15,0.15,0.07,0.06,0.03,0.03,0.00,0.00,0.01,-0.23,-0.22,-0.01,-0.01,-0.06,0.07,0.07,0.00,-0.01,-0.04,-0.04,Hot
178,0.11,-0.04,-0.04,0.00,0.04,-0.03,-0.03,0.01,0.04,0.00,0.00,0.01,-0.01,-0.01,0.00,-0.02,0.02,0.02,-0.01,-0.02,0.03,0.01,0.00,-0.05,0.00,0.55,0.55,-0.17,-0.18,-0.33,-0.33,-0.04,0.00,0.00,0.00,-0.04,0.01,0.02,0.01,1.54,1.54,-0.26,-0.27,-0.10,0.27,0.27,-0.39,-0.40,-0.44,-0.45,-0.01,0.00,

In [ ]:
#hot_test=diff_test.loc[diff_test.Class=='Hot',:].drop('Class',axis=1)
#cold_test=diff_test.loc[diff_test.Class=='Cold',:].drop('Class',axis=1)
#print(hot_test)
#print(cold_test)

In [ ]:
#hot_label=pycaret.regression.predict_model(best_hot_reg_model,hot_test)['Label']
#cold_label=pycaret.regression.predict_model(best_cold_reg_model,cold_test)['Label']
#label=pd.concat([hot_label,cold_label])
#label=label.sort_index()
#label

In [ ]:
submit=pd.DataFrame({'Day':range(1,181),'kW-12:00-23:50':pycaret.regression.predict_model(,diff_test)}).set_index('Day')
submit.to_csv('Submission.csv')